In [1]:
from __future__ import print_function

import os
import sys
import numpy as np

from keras.optimizers import SGD
from keras.callbacks import CSVLogger, ModelCheckpoint

sys.path.append(os.path.join(os.getcwd(), os.pardir))

import config

from utils.dataset.data_generator import DataGenerator
from models.cnn3_with_normalization import cnn_w_normalization

Using Theano backend.
Using gpu device 1: GeForce GTX 680 (CNMeM is disabled, cuDNN 5005)


In [2]:
lr=0.1
n_epochs=500
batch_size=32
input_shape=(140, 140, 3)

name = 'cnn_140_rgb_lr_%f_nesterov' % lr

In [3]:
print('loading model...')
model = cnn_w_normalization(input_shape=input_shape)
model.summary()

optimizer = SGD(lr=lr, clipnorm=1., clipvalue=0.5, nesterov=True)

print('compiling model...')
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print('done.')

csv_logger = CSVLogger('%s_training.log' % name)
best_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_best.hdf5" % name), verbose=1,
                                          save_best_only=True)

current_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_current.hdf5" % name), verbose=0)

loading model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 140, 140, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 140, 140, 128) 18944       input_1[0][0]                    
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 140, 140, 128) 0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 70, 70, 128)   0           activation_1[0][0]               
__________________________________________________________________________

In [4]:
print('Initializing data generators...')
train_data_gen = DataGenerator(dataset_file=config.train_data_file, batch_size=batch_size)
validation_data_gen = DataGenerator(dataset_file=config.validation_data_file, batch_size=batch_size)
test_data_gen = DataGenerator(dataset_file=config.test_data_file, batch_size=batch_size)
print('done.')

Initializing data generators...
done.


In [5]:
print('Fitting model...')
history = model.fit_generator(train_data_gen,
                              nb_epoch=n_epochs,
                              samples_per_epoch=train_data_gen.n_batches * batch_size,
                              validation_data=validation_data_gen,
                              nb_val_samples=validation_data_gen.n_samples,
                              verbose=1,
                              callbacks=[csv_logger, best_model_checkpointer, current_model_checkpointer])
print('done.')

Fitting model...
Epoch 1/500
10496/10496 [==============================] - 367s - loss: 0.6396 - acc: 0.6647 - val_loss: 0.6321 - val_acc: 0.6734
Epoch 2/500
10496/10496 [==============================] - 364s - loss: 0.6392 - acc: 0.6647 - val_loss: 0.6333 - val_acc: 0.6717
Epoch 3/500
10496/10496 [==============================] - 367s - loss: 0.6391 - acc: 0.6647 - val_loss: 0.6324 - val_acc: 0.6730
Epoch 4/500
10496/10496 [==============================] - 364s - loss: 0.6391 - acc: 0.6647 - val_loss: 0.6324 - val_acc: 0.6730
Epoch 5/500
10496/10496 [==============================] - 365s - loss: 0.6391 - acc: 0.6647 - val_loss: 0.6316 - val_acc: 0.6743
Epoch 6/500
10496/10496 [==============================] - 364s - loss: 0.6391 - acc: 0.6647 - val_loss: 0.6319 - val_acc: 0.6739
Epoch 7/500
10496/10496 [==============================] - 364s - loss: 0.6391 - acc: 0.6647 - val_loss: 0.6328 - val_acc: 0.6725
Epoch 8/500
10496/10496 [==============================] - 364s - loss: 0

In [6]:
print('Evaluating model...')
score = model.evaluate_generator(test_data_gen, val_samples=test_data_gen.n_samples)
print('done.')

print('Test score:', score[0])
print('Test accuracy:', score[1])

Evaluating model...
done.
Test score: 0.403033623813
Test accuracy: 0.908450704225
